<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/LinkedIn_Customer_Acquisition_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Pehlay required libraries install kar lein
%pip install --quiet -U langchain_google_genai langchain_core langgraph

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')
LANGCHAIN_API_KEY = userdata.get('langchain')

from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)

# Yeh do functions LinkedIn ke liye aik andaaza hain.
# In real-world scenario, aap ko LinkedIn API use karni hogi, OAuth tokens hasil karne hongay, waghera.
def find_linkedin_leads(query: str) -> str:
    """
    Yeh function LinkedIn par query ki buniyad par leads ki info wapis karega.
    Filhal sample data wapis ker rahe hain.
    """
    sample_leads = [
        {"name": "Ali Khan", "title": "Marketing Manager at ABC Company", "profile_url": "https://linkedin.com/in/alikhan"},
        {"name": "Sana Ahmed", "title": "Business Development at XYZ Inc.", "profile_url": "https://linkedin.com/in/sanaahmed"}
    ]
    # Normally: LinkedIn API call hoga. Yahan sirf json string return ho rahi hai.
    return f"Found leads for query '{query}': {sample_leads}"

def send_linkedin_message(profile_url: str, message: str) -> str:
    """
    Yeh function diye gaye profile_url par aik outreach message bhejta hai.
    Filhal yeh sirf pretend karega ke message chala gaya hai.
    """
    return f"Message sent to {profile_url}: {message}"

tools = [find_linkedin_leads, send_linkedin_message]

llm_with_tools = llm.bind_tools(tools)

from langgraph.graph import START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.graph.state import CompiledStateGraph
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langgraph.checkpoint.memory import MemorySaver

memory: MemorySaver = MemorySaver()

# System Prompt: Agent ki role definition
sys_msg = SystemMessage(content=(
    "You are a LinkedIn Customer Acquisition AI Agent. Your goal is to help the user find potential leads "
    "on LinkedIn, draft outreach messages, and provide strategies to improve customer acquisition. "
    "Always provide professional and strategic advice. If the user asks for specific lead searches, "
    "use the find_linkedin_leads tool. If they want to send a message, use the send_linkedin_message tool. "
    "Advise on best practices, personalization, and strategic outreach."
))

# Assistant node
def assistant(state: MessagesState) -> MessagesState:
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}

builder: StateGraph = StateGraph(MessagesState)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    tools_condition,
)
builder.add_edge("tools", "assistant")

graph = builder.compile(checkpointer=memory)

# Ab aap aik sample query chalain:
config = {"configurable": {"thread_id": "linkedintest1"}}

messages = graph.invoke({"messages": [HumanMessage(content="I need leads in the marketing sector in Karachi.")]}, config)
for m in messages['messages']:
    m.pretty_print()

# Aap iske baad user se outreach message draft karwane ke liye kahenge:
messages = graph.invoke({"messages": [HumanMessage(content="Can you draft a personalized outreach message for Ali Khan?")]}, config)
for m in messages['messages']:
    m.pretty_print()

# Is tarah aap business related scenario handle kar sakte hain.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.5/411.5 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 557.3 kB/s eta 0:00:00
================================ Human Message =================================

I need leads in the marketing sector in Karachi.
================================== Ai Message ==================================
Tool Calls:
  find_linkedin_leads (c102249b-d767-4460-8573-df08f385bc08)
 Call ID: c102249b-d767-4460-8573-df08f385bc08
  Args:
    query: marketing sector in Karachi
================================= Tool Message =================================
Name: find_linkedin_leads

Found leads for query 'marketing sector in Karachi': [{'name': 'Ali Khan', 'title': 'Marketing Manager at ABC Company', 'profile_url': 'https://linkedin.com/in/alikhan'}, {'name': 'Sana Ahmed', '

# Wazahat:

Upar wala code sirf aik sample hai. Actual LinkedIn integration ke liye LinkedIn’s API use karni hogi, authentication tokens hasil karne honge, aur real data parse karna hoga.

Tools ko real API calls se replace karke aap real-world scenario mein agent ko leads dhoondne aur messages bhejne ke qabil bana sakte hain.

Is approach se aap aik LinkedIn Customer Acquisition AI Agent bana sakte hain jo business aur clients ke liye qeemati ho sakta hai.